# Stop exits - e.g., stop loss & take profit

### - dummy test

In [102]:
import pandas
from datetime import datetime
import vectorbt as vbt


# mask variable is the signals for entries and exits
mask = pandas.DataFrame([   
    [True, False, False],
    [False, True, False],
    [False, False, True],
    [True, False, False],
    [False, True, False],
    [False, False, False]
], index=pandas.Index([
    datetime(2020, 1, 1),
    datetime(2020, 1, 2),
    datetime(2020, 1, 3),
    datetime(2020, 1, 4),
    datetime(2020, 1, 5),
    datetime(2020, 1, 6),
]), columns=['a', 'b', 'c'])


price = pandas.DataFrame({
    'open': [10, 11, 12, 11, 10, 10],
    'high': [11, 12, 13, 12, 11, 13],
    'low': [9, 10, 11, 10, 9, 10],
    'close': [10, 11, 12, 11, 10, 11]
})
out_dict_entries={}
out_dict_exits={}

In [103]:
# buy limit order => buy at next close price if next close price is 4% lower than current close price
entries = mask.vbt.signals.generate_ohlc_stop_exits(
    price["open"], price["high"], price["low"], price["close"],
    sl_stop=0.04, sl_trail=False,
    out_dict=out_dict_entries, exit_wait=1)

# sell limit order => exit trade at next close price if next close price is 8% higher than current close price
exits = mask.vbt.signals.generate_ohlc_stop_exits(
    price['open'], price['high'], price['low'], price['close'],
    tp_stop=0.08, 
    out_dict=out_dict_exits, exit_wait=1)

entries, exits = entries.vbt.signals.clean(exits) # remove ignored exit signals

In [104]:
entries

,a,b,c
2020-01-01,False,False,False
2020-01-02,False,False,False
2020-01-03,False,False,False
2020-01-04,False,True,True
2020-01-05,True,False,False
2020-01-06,False,False,False


In [107]:
from vectorbt.signals.enums import StopType
out_dict_entries["stop_type"].vbt(mapping=StopType).apply_mapping()

,a,b,c
2020-01-01,None,None,None
2020-01-02,None,None,None
2020-01-03,None,None,None
2020-01-04,None,StopLoss,StopLoss
2020-01-05,StopLoss,None,None
2020-01-06,None,None,None


In [108]:
out_dict_entries["stop_price"]

,a,b,c
2020-01-01,NaN,NaN,NaN
2020-01-02,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN
2020-01-04,NaN,10.56,11.52
2020-01-05,10.56,NaN,NaN
2020-01-06,NaN,NaN,NaN


In [109]:
pf = vbt.Portfolio.from_signals(price["close"],
                                entries,
                                exits,
                                open=price["open"],
                                high=price["high"],
                                low=price["low"],
                                init_cash=1e4,
                                direction="longonly",
                                fees=0.01, #in %
                                )
pf.trades.records_readable

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,a,990.099010,2020-01-05,10.0,99.009901,2020-01-06,11.0,108.910891,782.178218,0.079,Long,Closed,0
1,1,b,900.090009,2020-01-04,11.0,99.009901,2020-01-06,11.0,99.009901,-198.019802,-0.020,Long,Closed,1
2,2,c,900.090009,2020-01-04,11.0,99.009901,2020-01-06,11.0,99.009901,-198.019802,-0.020,Long,Closed,2


In [110]:
pf.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,a,2020-01-05,990.099010,10.0,99.009901,Buy
1,1,a,2020-01-06,990.099010,11.0,108.910891,Sell
2,2,b,2020-01-04,900.090009,11.0,99.009901,Buy
3,3,b,2020-01-06,900.090009,11.0,99.009901,Sell
4,4,c,2020-01-04,900.090009,11.0,99.009901,Buy
5,5,c,2020-01-06,900.090009,11.0,99.009901,Sell


In [111]:
pf["a"].stats()

Start                         2020-01-01 00:00:00
End                           2020-01-06 00:00:00
Period                            6 days 00:00:00
Start Value                               10000.0
End Value                            10782.178218
Total Return [%]                         7.821782
Benchmark Return [%]                         10.0
Max Gross Exposure [%]                      100.0
Total Fees Paid                        207.920792
Max Drawdown [%]                         0.990099
Max Drawdown Duration             1 days 00:00:00
Total Trades                                    1
Total Closed Trades                             1
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                100.0
Best Trade [%]                                7.9
Worst Trade [%]                               7.9
Avg Winning Trade [%]                         7.9
Avg Losing Trade [%]                          NaN


In [112]:

""" 
entries, exits = mask.vbt.signals.generate_ohlc_stop_exits(price["open"], price["high"], 
                                                           price["low"], price["close"],
                                                           sl_stop=0.04,sl_trail=False,
                                                           tp_stop=0.1,
                                                           chain=True, # remove ignored entry signals
                                                            out_dict=out_dict)
entries"""


' \nentries, exits = mask.vbt.signals.generate_ohlc_stop_exits(price["open"], price["high"], \n                                                           price["low"], price["close"],\n                                                           sl_stop=0.04,sl_trail=False,\n                                                           tp_stop=0.1,\n                                                           chain=True, # remove ignored entry signals\n                                                            out_dict=out_dict)\nentries'

## Real case

In [128]:
# Remark: close trades after 10 days holding it

# entries = <....>
# exits = entries.vbt.signals.fshift(10)
# entries, exits = entries.vbt.signals.clean(exits)

In [113]:
# Reference: stop exits with RANDENEX indicator: https://github.com/polakowo/vectorbt/issues/181


import vectorbt as vbt

data = vbt.YFData.download(
    "BTC-USD",
    start='2017-01-01 UTC',
    end='2020-01-01 UTC'
).concat()
order_price = data["Close"]
# Random enter signal generator based on the number of signals.
rand = vbt.RAND.run(data["Close"].shape, n=[10], seed=42)
# Random exit signal generator based on the number of signals.
randx = vbt.RANDX.run(rand.entries, seed=42)
pf = vbt.Portfolio.from_signals(data["Close"], rand.entries, randx.exits, price=order_price)  # without SL
pf.stats()

Start                         2017-01-01 00:00:00+00:00
End                           2019-12-31 00:00:00+00:00
Period                               1095 days 00:00:00
Start Value                                       100.0
End Value                                    442.020609
Total Return [%]                             342.020609
Benchmark Return [%]                         620.566853
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              49.380515
Max Drawdown Duration                 510 days 00:00:00
Total Trades                                         10
Total Closed Trades                                  10
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                       60.0
Best Trade [%]                                93.069372
Worst Trade [%]                              -19

In [114]:
out_dict = {}
stop_exits = rand.entries.vbt.signals.generate_ohlc_stop_exits(
    order_price, 
    data['High'],
    data['Low'], 
    data['Close'], 
    sl_stop=0.1, 
    tp_stop=0.1,
    out_dict=out_dict
)


In [115]:
out_dict

{'stop_price': Date
 2017-01-01 00:00:00+00:00   NaN
 2017-01-02 00:00:00+00:00   NaN
 2017-01-03 00:00:00+00:00   NaN
 2017-01-04 00:00:00+00:00   NaN
 2017-01-05 00:00:00+00:00   NaN
                              ..
 2019-12-27 00:00:00+00:00   NaN
 2019-12-28 00:00:00+00:00   NaN
 2019-12-29 00:00:00+00:00   NaN
 2019-12-30 00:00:00+00:00   NaN
 2019-12-31 00:00:00+00:00   NaN
 Freq: D, Length: 1095, dtype: float64,
 'stop_type': Date
 2017-01-01 00:00:00+00:00   -1
 2017-01-02 00:00:00+00:00   -1
 2017-01-03 00:00:00+00:00   -1
 2017-01-04 00:00:00+00:00   -1
 2017-01-05 00:00:00+00:00   -1
                             ..
 2019-12-27 00:00:00+00:00   -1
 2019-12-28 00:00:00+00:00   -1
 2019-12-29 00:00:00+00:00   -1
 2019-12-30 00:00:00+00:00   -1
 2019-12-31 00:00:00+00:00   -1
 Freq: D, Length: 1095, dtype: int32}

In [116]:
exits = randx.exits.vbt | stop_exits
order_price = order_price.copy()
order_price = out_dict["stop_price"]
pf2 = vbt.Portfolio.from_signals(data['Close'], rand.entries, exits, price=order_price)  # with SL
pf2.stats()

Start                         2017-01-01 00:00:00+00:00
End                           2019-12-31 00:00:00+00:00
Period                               1095 days 00:00:00
Start Value                                       100.0
End Value                                         100.0
Total Return [%]                                    0.0
Benchmark Return [%]                         620.566853
Max Gross Exposure [%]                              0.0
Total Fees Paid                                     0.0
Max Drawdown [%]                                    NaN
Max Drawdown Duration                               NaT
Total Trades                                          0
Total Closed Trades                                   0
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                        NaN
Best Trade [%]                                      NaN
Worst Trade [%]                                 

In [117]:
import pandas
df = pandas.DataFrame(rand.entries)
df

,10
0,False
1,False
2,False
3,False
4,False
...,...
1090,False
1091,False
1092,False
1093,False


In [118]:
rand.entries.vbt.signals.fshift(10)

0       False
1       False
2       False
3       False
4       False
        ...  
1090    False
1091    False
1092    False
1093    False
1094    False
Name: 10, Length: 1095, dtype: bool

In [119]:
stop_exits = rand.entries.vbt.signals.generate_ohlc_stop_exits(
    data["Open"], 
    data['High'],
    data['Low'], 
    data['Close'], 
    sl_stop=0.1,
    tp_stop=0.1,
    out_dict=out_dict
)

stop_entries = randx.exits.vbt.signals.generate_ohlc_stop_exits(
    data["Open"], 
    data['High'],
    data['Low'], 
    data['Close'], 
    sl_stop=0.1,
    tp_stop=0.1,
)

new_entries = rand.entries.vbt | stop_entries
new_exits = randx.exits.vbt | stop_exits
new_entries, new_exits = new_entries.vbt.signals.clean(new_exits)

order_price =out_dict["stop_price"]
pf4= vbt.Portfolio.from_signals(data['Close'], new_entries, new_exits,
                                price = order_price)
pf4.stats()

Start                         2017-01-01 00:00:00+00:00
End                           2019-12-31 00:00:00+00:00
Period                               1095 days 00:00:00
Start Value                                       100.0
End Value                                         100.0
Total Return [%]                                    0.0
Benchmark Return [%]                         620.566853
Max Gross Exposure [%]                              0.0
Total Fees Paid                                     0.0
Max Drawdown [%]                                    NaN
Max Drawdown Duration                               NaT
Total Trades                                          0
Total Closed Trades                                   0
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                        NaN
Best Trade [%]                                      NaN
Worst Trade [%]                                 

In [120]:
pf3 = vbt.Portfolio.from_signals(data['Close'], rand.entries, randx.exits, 
                                    sl_stop=0.1, tp_stop=0.1,
                                    stop_entry_price="price",
                                    open=data["Open"], high=data["High"], low=data["Low"]
                                   )  # with SL
pf3.stats()

Start                         2017-01-01 00:00:00+00:00
End                           2019-12-31 00:00:00+00:00
Period                               1095 days 00:00:00
Start Value                                       100.0
End Value                                      106.4542
Total Return [%]                                 6.4542
Benchmark Return [%]                         620.566853
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                                   19.0
Max Drawdown Duration                 555 days 00:00:00
Total Trades                                         10
Total Closed Trades                                  10
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                       60.0
Best Trade [%]                                     10.0
Worst Trade [%]                                 

In [121]:
pf.trades.records_readable

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,0,0.097425,2017-03-30 00:00:00+00:00,1026.430054,0.0,2017-04-13 00:00:00+00:00,1169.280029,0.0,13.917166,0.139172,Long,Closed,0
1,1,0,0.054644,2017-05-20 00:00:00+00:00,2084.729980,0.0,2017-06-10 00:00:00+00:00,2947.709961,0.0,47.156339,0.413953,Long,Closed,1
2,2,0,0.027620,2017-10-14 00:00:00+00:00,5831.790039,0.0,2018-01-25 00:00:00+00:00,11259.400391,0.0,149.910099,0.930694,Long,Closed,2
3,3,0,0.046175,2018-06-18 00:00:00+00:00,6734.819824,0.0,2018-07-03 00:00:00+00:00,6529.589844,0.0,-9.476595,-0.030473,Long,Closed,3
4,4,0,0.044952,2018-08-26 00:00:00+00:00,6707.259766,0.0,2018-09-15 00:00:00+00:00,6543.200195,0.0,-7.374861,-0.024460,Long,Closed,4
5,5,0,0.082951,2018-12-17 00:00:00+00:00,3545.864746,0.0,2019-04-19 00:00:00+00:00,5303.812500,0.0,145.823089,0.495774,Long,Closed,5
6,6,0,0.081483,2019-04-22 00:00:00+00:00,5399.365234,0.0,2019-05-25 00:00:00+00:00,8052.543945,0.0,216.188351,0.491387,Long,Closed,6
7,7,0,0.063331,2019-09-13 00:00:00+00:00,10360.546875,0.0,2019-09-24 00:00:00+00:00,8620.566406,0.0,-110.194669,-0.167943,Long,Closed,7
8,8,0,0.058002,2019-11-04 00:00:00+00:00,9412.612305,0.0,2019-12-08 00:00:00+00:00,7564.345215,0.0,-107.202909,-0.196361,Long,Closed,8
9,9,0,0.060609,2019-12-26 00:00:00+00:00,7238.966797,0.0,2019-12-30 00:00:00+00:00,7292.995117,0.0,3.274599,0.007464,Long,Closed,9
